# The Reuters Dataset

---

### Colab Note

Don't forget that you can link your notebook to your drive and save your work there. Then you can download and backup your models, reload them to keep training them, or upload datasets to your drive. 

In [ ]:
import os
import sys

if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('drive/My Drive/') # 'My Drive' is the default name of Google Drives
    os.listdir()
    
# use os.chdir("my-directory") # to change directory, and
# os.listdir()                 # to list its contents
# os.getcwd()                  # to get the name of the current directory
# os.mkdir("my-new-dir")       # to create a new directory
# See: https://realpython.com/working-with-files-in-python/

# You can also use bash commands directly, preceded by a bang
# !ls
# However, the following will *not* change the Python directory 
# the notebook points to (use os.chdir for that)!
# !cd my-directory    

---

## 1. Practice

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

### For reproducible results

In Keras ([source](https://keras.io/examples/keras_recipes/reproducibility_recipes/)):
```python
tf.keras.utils.set_random_seed(812) # See below

# If using TensorFlow, this will make GPU ops as deterministic as possible,
# but it will affect the overall performance, so be mindful of that.
tf.config.experimental.enable_op_determinism()
```

Note: `tf.keras.utils.set_random_seed` will do the following ([source](https://github.com/keras-team/keras/blob/f6c4ac55692c132cd16211f4877fac6dbeead749/keras/src/utils/rng_utils.py#L10)):

```python
import random
random.seed(42)

import numpy as np
np.random.seed(42)

tf.random.set_seed(42) # can be any number
```

### Dataset processing & model building

In [ ]:
def vectorize_sequences(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

# load
(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.reuters.load_data(num_words=10000)

# preprocess
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

one_hot_train_labels = tf.keras.utils.to_categorical(train_labels)
one_hot_test_labels = tf.keras.utils.to_categorical(test_labels)

# split training set into train & validation
partial_x_train=x_train[1000:]
partial_y_train=one_hot_train_labels[1000:]
x_val=x_train[:1000]
y_val=one_hot_train_labels[:1000]

In [ ]:
# build
model = tf.keras.models.Sequential()
model.add(tf.keras.Input((10000,)))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(46, activation = 'softmax'))

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

### Baselines

1) Do you remember the baseline discussion in the lecture? You can import the code and test it here.

2) Sanity check, how does our network perform before training (use `.evaluate` on `partial_x_train, partial_y_train`). Is the accuracy a value you would expect?

### Training

In [ ]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=512,
    validation_data=(x_val,y_val)
)

### Visualise your results

Import `matplotlib` code to visualise the result of your experiments (from lab 3 or the lectures).

Can you think of ways to make the code easy to reuse (when running several experiments)?

As usual, we are after the epoch where best *validation* result (accuracy) has been achieved. Can you think of a way to retrieve that optimal epoch programmatically?

### Experiments

- Plot training and validation loss and accuracy. Can you spot the optimal epoch (where the best *validation accuracy* has been achieved)?
- Deliberately cause an information bottleneck by building a 64-4-46 network. Compare the validation accuracy with a 64-64-46 network;
- Try using larger layers – 128 units and smaller layers – 32 units;
- Try less and more hidden layers;
- Tabulate your results / organise your experiments as much as you can.

## 2. Conclusion

Take your best network and train on **all the training data** (`x_train`, `one_hot_train_labels`), without a train/validation split, using the same hyperparameters (optimizer, learning rate, network size, etc.) as your best run, for the optimal number of epochs (looking at your best validation curves).

Evaluate this last model on the test set (`x_test, one_hot_test_labels`).

### Use your model (optional) 

Can you import the lecture code used to test the model on a newswire, and see if you agree with its prediction?

AS shown in the lecture, the 46 topics are:
```python
# https://github.com/keras-team/keras/issues/12072
# and here: https://martin-thoma.com/nlp-reuters/
topics = [
    'cocoa','grain','veg-oil','earn','acq','wheat','copper','housing','money-supply',
    'coffee','sugar','trade','reserves','ship','cotton','carcass','crude','nat-gas',
    'cpi','money-fx','interest','gnp','meal-feed','alum','oilseed','gold','tin',
    'strategic-metal','livestock','retail','ipi','iron-steel','rubber','heat','jobs',
    'lei','bop','zinc','orange','pet-chem','dlr','gas','silver','wpi','hog','lead'
]
```

### Save and load models

To save and load models locally, you can use [the high-level API](https://www.tensorflow.org/tutorials/keras/save_and_load):
```python
model.save("my_reuters_model.keras")
```
Later one, to reload it, use:
```python
reloaded_model = tf.keras.models.load_model('my_reuters_model.keras')
```

It is also possible to save not just the model, but also the state of your optimiser, and every variable used during training, using the more involved [checkpoints](https://www.tensorflow.org/guide/checkpoint#create_the_checkpoint_objects).

## 3. Additional experiments

- Like in the IMDB dataset, one parameter you could study is the influence on the vocabulary size on the final results: you might then want to store the vocab size in a variable, and use that instead of the hard-coded `10000` that we have.
- Another line of enquiry is the study of the behaviour of your trained model:
  - Are you able to modify existing reviews in a way that changes the initial prediction of your model? (One 'automated' way of doing that would be to remove a certain number of words from the review, and see how performance is impacted by that information loss.)
  - Are you able to create a pipeline where you write your own review, or find one online, transform it into the appropriate format (remove punctuation, turn everything to lower case, convert to an array of integers using the dictionary yielded by `tf.keras.datasets.imdb.get_word_index()` (beware of the shift by 3 induced by the reserved tokens for padding, start of sequence and unknown!), and see what prediction you get for it?